In [ ]:
import pysam

ak1 = pysam.AlignmentFile("chr20_500000-1000000_AK1.bam", 'rb')
a = next(ak1)
b = next(ak1)

a.get_tag('MM')
a.get_tag('ML')

#Negative 일 경우
for pos, prob in list(a.modified_bases.values())[0]:
    if a.seq[pos-1: pos+1] != 'CG':
        print('fuck')
#Positive 일 경우
for pos, prob in list(b.modified_bases.values())[0]:
    if b.seq[pos: pos+2] != 'CG':
        print('fuck')   